# Demo to test and show the functionality of the library

In [1]:
# Importing Functions
import numpy as np
import matplotlib.pyplot as plt
import matplotlib
matplotlib.use('TKAgg') # Needed to have figures display properly. 
import flirimageextractor
import Utils as u
import cv2

DEBUG=False

In [2]:
filename = "C:\\Users\\Francisco\\Documents\\dataset_termal_destino\\Control_Image termal C_paciente 3 (VC)_flir_control_flir_20190907T092339.jpg"
#filename = "D:\\FLIROne\FLIR_20230426_044637_061.jpg"
#filename = "D:\\FLIROne\FLIR_20230426_044706_939.jpg"
flir = flirimageextractor.FlirImageExtractor(exiftool_path="C:\\Windows\\exiftool.exe")
flir.process_image(filename, RGB=True)

In [3]:
therm = flir.get_thermal_np()
rgb_fullres = flir.get_rgb_np()
if DEBUG:
    plt.figure(figsize=(10,5))
    plt.subplot(1,2,1)
    plt.imshow(therm)
    plt.title('Raw Thermal Image')
    plt.subplot(1,2,2)
    plt.imshow(rgb_fullres)
    plt.title('RGB Full Resolution Image')
    plt.show(block='TRUE') 

In [4]:
colormap_image, Imsalida,temp_img_resized,image_copy = u.extract_images(flir,plot=0)

Image with offsetx=-1, offsety=37, pipx2=479, pipy2=639, real2ir=1.22885632514954


In [5]:
#u.save_thermal_csv(temp_img_resized,"temperature_examples.csv",delimiter=";")

# second part of the demo get the perspective transformation between two images

In [6]:
filename2 = "C:\\Users\\Francisco\\Documents\\dataset_termal_destino\\Control_Image termal C_paciente 3 (VC)_flir_control_flir_20190907T092409.jpg"
flir2 = flirimageextractor.FlirImageExtractor(exiftool_path="C:\\Windows\\exiftool.exe")
flir2.process_image(filename2, RGB=True)
therm2 = flir2.get_thermal_np()
rgb_fullres2 = flir2.get_rgb_np()
colormap_image2, Imsalida2,temp_img_resized2,image_copy2 = u.extract_images(flir2,plot=0)
#u.save_thermal_csv(temp_img_resized2,"temperature_examples2.csv",delimiter=";")
if DEBUG:
    plt.figure(figsize=(10,5))
    plt.subplot(1,2,1)
    plt.imshow(temp_img_resized2)
    plt.title('temp_img_resized2')
    plt.subplot(1,2,2)
    plt.imshow(image_copy2)
    plt.title('image_copy2')
    plt.show(block='TRUE') 


Image with offsetx=-1, offsety=37, pipx2=479, pipy2=639, real2ir=1.22885632514954


In [7]:


"""
def fethearing_bin_to_color(binary_image, color_image):
    # Operación de cierre morfológico con un kernel elíptico de tamaño 5x5
    kernel = cv2.getStructuringElement(cv2.MORPH_ELLIPSE, (7, 7))
    closed_image = cv2.morphologyEx(binary_image, cv2.MORPH_CLOSE, kernel)
    
    # Fethearing usando filtro guiado
    filtered_image = cv2.ximgproc.guidedFilter(color_image, closed_image, 20, eps=0.1)#5, eps=1e-4
    
    # Umbralización de la imagen filtrada
    _, thresholded_image = cv2.threshold(filtered_image,20, 255, cv2.THRESH_BINARY)
    
    return filtered_image
"""


'\ndef fethearing_bin_to_color(binary_image, color_image):\n    # Operación de cierre morfológico con un kernel elíptico de tamaño 5x5\n    kernel = cv2.getStructuringElement(cv2.MORPH_ELLIPSE, (7, 7))\n    closed_image = cv2.morphologyEx(binary_image, cv2.MORPH_CLOSE, kernel)\n    \n    # Fethearing usando filtro guiado\n    filtered_image = cv2.ximgproc.guidedFilter(color_image, closed_image, 20, eps=0.1)#5, eps=1e-4\n    \n    # Umbralización de la imagen filtrada\n    _, thresholded_image = cv2.threshold(filtered_image,20, 255, cv2.THRESH_BINARY)\n    \n    return filtered_image\n'

In [8]:
import registration_simpleitk as sr

In [9]:
segmented_Feet2,segmented_temps2=u.Find_feets(image_copy2,temp_img_resized2,percentage=20)
print(segmented_Feet2[1].shape)
if DEBUG:
    u.plot_images_and_thermal(segmented_Feet2,segmented_temps2)
segmented_Feet,segmented_temps=u.Find_feets(image_copy,temp_img_resized,percentage=20)
if DEBUG:
    u.plot_images_and_thermal(segmented_Feet,segmented_temps)


(441, 204, 3)


In [10]:
#u.visualize_descriptors(segmented_Feet[0],segmented_Feet2[0])
#u.visualize_descriptors(segmented_Feet[1],segmented_Feet2[1])

"""
output_image,homography_matrix=u.matchbydescriptors(segmented_Feet[0],segmented_Feet2[0])
plt.figure(figsize=(10,5))
plt.imshow(output_image)
plt.title('registration between images')
plt.show(block='TRUE') 

output_image,homography_matrix=u.matchbydescriptors(segmented_Feet[1],segmented_Feet2[1])
plt.figure(figsize=(10,5))
plt.imshow(output_image)
plt.title('registration between images')
plt.show(block='TRUE') 
"""

"\noutput_image,homography_matrix=u.matchbydescriptors(segmented_Feet[0],segmented_Feet2[0])\nplt.figure(figsize=(10,5))\nplt.imshow(output_image)\nplt.title('registration between images')\nplt.show(block='TRUE') \n\noutput_image,homography_matrix=u.matchbydescriptors(segmented_Feet[1],segmented_Feet2[1])\nplt.figure(figsize=(10,5))\nplt.imshow(output_image)\nplt.title('registration between images')\nplt.show(block='TRUE') \n"

In [11]:
# using a different approach
registered_image_a_array, registered_image_b_array, composition_image =sr.register_images(segmented_Feet[0],segmented_Feet2[0],'displaceMeth4.hdf5')
if DEBUG:
    plt.figure(figsize=(10,5))
    plt.subplot(1,2,1)
    plt.imshow(registered_image_a_array)
    plt.title('registered_image_a_array')
    plt.subplot(1,2,2)
    plt.imshow(registered_image_b_array)
    plt.title('registered_image_b_array')
    plt.show(block='TRUE') 
    
plt.figure(figsize=(10,5))
plt.imshow(composition_image)
plt.title('composition_image')
plt.show(block='TRUE') 


  0 =   -1.15621 : (-0.2289803080446606, -0.44448624110063556)
  1 =   -1.18144 : (-0.4609793312485472, -0.8874043523045493)
  2 =   -1.19330 : (-0.9166324520064172, -0.681539752279159)
  3 =   -1.19062 : (-0.7817573792395652, -0.8920361055638095)
  4 =   -1.19219 : (-0.6868501636292976, -0.8106873475824722)
  5 =   -1.19278 : (-0.590082666187128, -0.889814153652095)
  6 =   -1.19334 : (-0.5664777718912027, -0.8319430990109518)
  7 =   -1.19333 : (-0.5642483176615282, -0.8631134701001414)
  8 =   -1.19338 : (-0.5563369265500226, -0.84963940183251)
  9 =   -1.19339 : (-0.556429023761788, -0.8574513589722788)
-------
itk::simple::TranslationTransform
 TranslationTransform (000001848DF37830)
   RTTI typeinfo:   class itk::TranslationTransform<double,2>
   Reference Count: 2
   Modified Time: 20531
   Debug: Off
   Object Name: 
   Observers: 
     none
   Offset: [-0.556429, -0.857451]

Optimizer stop condition: RegularStepGradientDescentOptimizerv4: Gradient magnitude tolerance met after

In [12]:
# using a different approach
output =sr.register_images_Similarity2DTransform(segmented_Feet[0],segmented_Feet2[0],'displaceMeth4_advanced.hdf5')
fixed_imagen = sr.sitk_image_to_opencv(output["fixed"])
moving_imagen = sr.sitk_image_to_opencv(output["moving"])
composition_image = sr.sitk_image_to_opencv(output["composition"])
if DEBUG:
    plt.figure(figsize=(10,5))
    plt.subplot(1,2,1)
    plt.imshow(fixed_imagen)
    plt.title('fixed_imagen')
    plt.subplot(1,2,2)
    plt.imshow(moving_imagen)
    plt.title('moving_imagen')
    plt.show(block='TRUE') 

plt.figure(figsize=(10,5))
plt.imshow(composition_image)
plt.title('composition_image')
plt.show(block='TRUE') 


  0 =   -1.18712 : (0.9988617177801171, 0.0003563966385609195, -1.099609511610336, -0.5087477135169298)
  1 =   -1.19015 : (0.997330001746463, 0.0008698446840783218, -1.0942575255143294, -0.5584341951988214)
  2 =   -1.19618 : (0.9971108499566993, 0.0009682151618311212, -1.0590011492093128, -0.5938874069797194)
  3 =   -1.19867 : (0.9968908303538213, 0.0010813195633493158, -1.0288649748664278, -0.6337841461289775)
  4 =   -1.20106 : (0.9966654843860195, 0.0011852849372778061, -0.9922932400967068, -0.6678788176529707)
  5 =   -1.20396 : (0.9964396449484639, 0.0012770868871260221, -0.9528237825652517, -0.6985725057846226)
  6 =   -1.20653 : (0.9961948508633904, 0.0013764164364051988, -0.9156864182721576, -0.732050056215599)
  7 =   -1.20839 : (0.995892212760326, 0.001540184374604116, -0.8990105212615724, -0.779185987829744)
  8 =   -1.20926 : (0.9958831198621334, 0.0015855313530696111, -0.923719732153602, -0.7829876661517425)
  9 =   -1.20942 : (0.9958147128127952, 0.0016588900231014859,

 23 =   -1.21078 : (0.9945232053353834, 0.002399045333651023, -1.1649754131347572, -0.7519878366098993)
 24 =   -1.21079 : (0.9945040022665125, 0.0024041031895930248, -1.171203606216349, -0.7525091010328665)
 25 =   -1.21080 : (0.9944119267964593, 0.002411464576951669, -1.176678277135202, -0.7494954742729549)
 26 =   -1.21083 : (0.9944221101493609, 0.0024140164027080945, -1.1826274739841778, -0.7514110542477007)
 27 =   -1.21083 : (0.9944135927158525, 0.0024151798208489057, -1.18881984186396, -0.7522578163992092)
 28 =   -1.21083 : (0.9943939674777202, 0.002413330545255802, -1.1860160020391757, -0.7508780632810167)
 29 =   -1.21083 : (0.9943953136717818, 0.0024137718978603415, -1.1875194252849184, -0.751303648510001)
 30 =   -1.21083 : (0.9943943118813995, 0.0024140042628181288, -1.18904969726627, -0.751619358591425)
-------
itk::simple::Similarity2DTransform
 Similarity2DTransform (000001848DE86BC0)
   RTTI typeinfo:   class itk::Similarity2DTransform<double>
   Reference Count: 3
   

In [13]:

print(type(fixed_imagen))

print(fixed_imagen.shape)

print(fixed_imagen)

<class 'numpy.ndarray'>
(206, 445)
[[40 40 40 ... 13 13 13]
 [40 40 40 ... 13 13 13]
 [39 39 40 ... 13 13 13]
 ...
 [44 45 43 ... 18 17 16]
 [44 45 43 ... 17 16 16]
 [45 45 43 ... 18 17 16]]
